# Fine-Tuning Qwen2-VL for Weather Satellite Image Analysis

## Overview
This notebook demonstrates the fine-tuning of the **Qwen2-VL** (Vision-Language) model for weather satellite image analysis using Parameter-Efficient Fine-Tuning (PEFT) techniques.

### Key Features
- **Model**: Qwen2-VL-7B-Instruct
- **Techniques**: LoRA, DoRA, and AdaLoRA
- **Dataset**: Weather Satellite Images with descriptive captions
- **Framework**: HuggingFace Transformers + TRL + PEFT

### Project Structure
1. 🔧 Environment Setup & Dependencies
2. 📊 Dataset Loading & Preparation  
3. 🤖 Model Configuration & Fine-Tuning
4. 📈 Evaluation & Testing
5. 🚀 Model Export & Deployment

---

## 1. Environment Setup & Dependencies

### Install Required Libraries
Installing all necessary packages for model fine-tuning, including:
- `bitsandbytes`: For 4-bit quantization
- `qwen-vl-utils`: Qwen vision-language utilities
- `trl`: Transformer Reinforcement Learning library
- `peft`: Parameter-Efficient Fine-Tuning

In [ ]:
!pip install -q -U bitsandbytes qwen-vl-utils trl

## 2. Dataset Loading & Preparation

### Download Dataset from Kaggle
Loading the weather satellite image dataset from Kaggle Hub.

In [ ]:
import kagglehub

dataset_path = kagglehub.dataset_download('azdinsahir/wsi-data-pfe')

print('Data source import complete.')


Data source import complete.


## 3. Model Configuration & Fine-Tuning

### Complete Fine-Tuning Pipeline
This cell contains the complete fine-tuning implementation including:
- Model loading with 4-bit quantization
- LoRA configuration
- Training arguments setup
- Dataset preparation
- Model training and evaluation
- Model saving and testing

In [ ]:
# Main Fine-Tuning Script
"""
Simple Qwen2-VL LoRA Fine-tuning for Weather Satellite Dataset
Based exactly on the working HuggingFace notebook - MINIMAL AND FUNCTIONAL
"""

import os
import json
import torch
import wandb
import gc
import time
from PIL import Image
from transformers import Qwen2VLForConditionalGeneration, Qwen2VLProcessor, BitsAndBytesConfig
from peft import LoraConfig, get_peft_model
from trl import SFTConfig, SFTTrainer
from qwen_vl_utils import process_vision_info
!pip install -q qwen-vl-utils trl --q
def clear_memory():
    """Clear GPU memory (exactly from notebook)"""
    # Delete variables if they exist in the current global scope
    if 'inputs' in globals(): del globals()['inputs']
    if 'model' in globals(): del globals()['model']
    if 'processor' in globals(): del globals()['processor']
    if 'trainer' in globals(): del globals()['trainer']
    if 'peft_model' in globals(): del globals()['peft_model']
    if 'bnb_config' in globals(): del globals()['bnb_config']
    time.sleep(2)

    # Garbage collection and clearing CUDA memory
    gc.collect()
    time.sleep(2)
    torch.cuda.empty_cache()
    torch.cuda.synchronize()
    time.sleep(2)
    gc.collect()
    time.sleep(2)

    print(f"GPU allocated memory: {torch.cuda.memory_allocated() / 1024**3:.2f} GB")
    print(f"GPU reserved memory: {torch.cuda.memory_reserved() / 1024**3:.2f} GB")

def load_weather_dataset(dataset_path, max_samples=None):
    """Load weather satellite dataset and format with METADATA like GPT-4V collection"""

    print(f"📂 Loading weather dataset from: {dataset_path}")

    # Load manifest
    manifest_path = os.path.join(dataset_path, "training_manifest.json")
    with open(manifest_path, 'r') as f:
        manifest = json.load(f)

    # Use all samples or limit
    if max_samples is None:
        data = manifest['training_data']
        print(f"Using ALL {len(data)} samples")
    else:
        data = manifest['training_data'][:max_samples]
        print(f"Limited to {len(data)} samples (max_samples={max_samples})")

    images_dir = os.path.join(dataset_path, "images")
    captions_dir = os.path.join(dataset_path, "captions")
    metadata_dir = os.path.join(dataset_path, "metadata")

    print(f"Processing {len(data)} samples with metadata integration...")

    def extract_temporal_context(metadata):
        """Extract temporal information from metadata (same as GPT-4V script)"""
        temporal_info = {}

        if 'temporal_info' in metadata:
            temp_data = metadata['temporal_info']
            temporal_info = {
                'date': temp_data.get('date_formatted', 'Unknown date'),
                'season': temp_data.get('season', 'Unknown season'),
                'month': temp_data.get('month_name', 'Unknown month')
            }

        if 'image_info' in metadata and 'capture_date' in metadata['image_info']:
            temporal_info['date'] = metadata['image_info']['capture_date']

        return temporal_info

    def extract_weather_context(metadata):
        """Extract weather information (same as GPT-4V script)"""
        weather_info = []

        if 'weather_descriptions' in metadata:
            for region_key, region_data in metadata['weather_descriptions'].items():
                raw_data = None

                if 'raw_data' in region_data:
                    raw_data = region_data['raw_data']
                elif 'regional_weather' in metadata and region_key in metadata['regional_weather']:
                    raw_data = metadata['regional_weather'][region_key].get('weather_12utc', {})

                weather_entry = {
                    'region': region_key,
                    'location': region_data.get('location', region_key),
                    'description': region_data.get('description', 'No description'),
                    'raw_data': raw_data or {}
                }
                weather_info.append(weather_entry)

        return weather_info

    def extract_city_coordinates(metadata):
        """Extract city coordinates (same as GPT-4V script)"""
        cities_info = []
        if 'regional_weather' in metadata:
            for city_key, city_data in metadata['regional_weather'].items():
                if 'coordinates' in city_data:
                    coords = city_data['coordinates']
                    location_desc = city_data.get('location', city_key)
                    cities_info.append({
                        'name': city_key.replace('_', ' ').title(),
                        'description': location_desc,
                        'lat': coords['lat'],
                        'lon': coords['lon']
                    })
        return cities_info

    def format_weather_data(weather_context):
        """Format weather data table (same as GPT-4V script)"""
        if not weather_context:
            return "Weather data unavailable"

        weather_table = "**SURFACE METEOROLOGICAL CONDITIONS (at 12:00 Noon UTC):**\n"
        weather_table += "| Location | Temp(°C) | Humidity(%) | Wind(m/s) | Precip(mm) | Cloud(%) | Pressure(kPa) |\n"
        weather_table += "|----------|----------|-------------|-----------|------------|----------|---------------|\n"

        for region_data in weather_context:
            if isinstance(region_data, dict) and 'raw_data' in region_data:
                raw = region_data['raw_data']
                location = region_data.get('location', 'Unknown').split(',')[0]
                temp = f"{raw.get('T2M', 'N/A'):.1f}" if raw.get('T2M') != 'N/A' else 'N/A'
                humidity = f"{raw.get('RH2M', 'N/A'):.0f}" if raw.get('RH2M') != 'N/A' else 'N/A'
                wind = f"{raw.get('WS10M', 'N/A'):.1f}" if raw.get('WS10M') != 'N/A' else 'N/A'
                precip = f"{raw.get('PRECTOTCORR', 'N/A'):.1f}" if raw.get('PRECTOTCORR') != 'N/A' else 'N/A'
                cloud = f"{raw.get('CLOUD_AMT', 'N/A'):.0f}" if raw.get('CLOUD_AMT') != 'N/A' else 'N/A'
                pressure = f"{raw.get('PS', 'N/A'):.0f}" if raw.get('PS') != 'N/A' else 'N/A'

                weather_table += f"| {location} | {temp} | {humidity} | {wind} | {precip} | {cloud} | {pressure} |\n"

        return weather_table

    def build_enhanced_system_prompt(metadata):
        """Build the SAME enhanced prompt used in GPT-4V collection"""

        temporal_context = extract_temporal_context(metadata)
        weather_context = extract_weather_context(metadata)
        weather_table = format_weather_data(weather_context)
        cities_info = extract_city_coordinates(metadata)

        # Build city reference section (cities only)
        city_references = "**MOROCCAN CITIES (for geographic reference):**\n"
        for city in cities_info:
            city_name = city['name'].replace('_', ' ').title()
            if not any(word in city_name.lower() for word in ['mountains', 'sahara', 'desert', 'atlas', 'rif']):
                city_references += f"- **{city_name}** ({city['lat']:.2f}°N, {city['lon']:.2f}°W): {city['description']}\n"

        date_info = temporal_context.get('date', 'Unknown date')
        season_info = temporal_context.get('season', 'Unknown season')
        month_info = temporal_context.get('month', 'Unknown month')

        # EXACT SAME PROMPT as used in GPT-4V collection
        system_prompt = f"""You are a professional meteorologist and satellite imagery analyst with expertise in North African climatology and Morocco's atmospheric patterns, trained in international meteorological standards (WMO/NOAA/EUMETSAT protocols).

**SATELLITE ACQUISITION DETAILS:**
- **Date & Time**: {date_info} at 12:00 Noon UTC (Midday acquisition)
- **Geographic Domain**: Morocco, North Africa
- **Seasonal Context**: {season_info} ({month_info})
- **Satellite**: MODIS Terra - Corrected Reflectance True Color
- **Spatial Resolution**: 250 meters

{weather_table}

{city_references}

**ANALYSIS FRAMEWORK:**
Following standard meteorological satellite interpretation protocols for comprehensive weather analysis including:

1. **TEMPORAL AND GEOGRAPHIC CONTEXT:** Reference exact acquisition date, noon timing, and specific Moroccan cities
2. **SURFACE METEOROLOGICAL CONDITIONS:** Integrate ground-level weather measurements with satellite observations
3. **ATMOSPHERIC PHENOMENA IDENTIFICATION:** Cloud field analysis, weather systems, and dust analysis if visible
4. **CITY-FOCUSED WEATHER ANALYSIS:** Link atmospheric conditions to specific cities with ground-level measurements
5. **METEOROLOGICAL INTERPRETATION:** Atmospheric stability assessment and moisture distribution patterns

**CRITICAL INSTRUCTIONS:**
- ALWAYS begin with date and 12:00 noon UTC acquisition time
- ALWAYS reference specific Moroccan CITIES by name (not regions)
- ALWAYS specify "ground-level" or "surface" when mentioning weather measurements
- Keep descriptions concise, professional, and focused on observable atmospheric phenomena

Provide comprehensive meteorological analysis focusing on cities and ground-level weather integration."""

        return system_prompt

    def format_weather_sample(item):
        """Format sample with METADATA exactly like GPT-4V collection"""

        # Load image
        image_path = os.path.join(images_dir, item['image_file'])
        image = Image.open(image_path).convert('RGB')

        # Load caption
        caption_path = os.path.join(captions_dir, item['caption_file'])
        with open(caption_path, 'r', encoding='utf-8') as f:
            caption = f.read().strip()

        # Load metadata and create enhanced system prompt
        enhanced_system_message = """You are an expert meteorologist analyzing weather satellite imagery. Provide detailed, professional analysis of atmospheric conditions visible in MODIS Terra satellite images of Morocco."""

        if item.get('has_metadata') and item.get('metadata_file'):
            metadata_path = os.path.join(metadata_dir, item['metadata_file'])
            try:
                with open(metadata_path, 'r') as f:
                    metadata = json.load(f)

                # Use the SAME enhanced prompt as GPT-4V collection
                enhanced_system_message = build_enhanced_system_prompt(metadata)

            except Exception as e:
                print(f"⚠️ Could not load metadata for {item['image_file']}: {e}")

        # Format exactly like ChartQA but with enhanced system prompt
        return [
            {
                "role": "system",
                "content": [
                    {
                        "type": "text",
                        "text": enhanced_system_message
                    }
                ],
            },
            {
                "role": "user",
                "content": [
                    {
                        "type": "image",
                        "image": image,
                    },
                    {
                        "type": "text",
                        "text": "Please analyze this MODIS Terra satellite image of Morocco and provide a comprehensive meteorological analysis using the provided weather data and geographic context.",
                    }
                ],
            },
            {
                "role": "assistant",
                "content": [
                    {
                        "type": "text",
                        "text": caption
                    }
                ],
            },
        ]

    # Format all samples with progress tracking
    formatted_data = []
    for i, item in enumerate(data):
        try:
            sample = format_weather_sample(item)
            formatted_data.append(sample)
            if i % 50 == 0:
                print(f"  Processed {i}/{len(data)} samples...")
        except Exception as e:
            print(f"  ⚠️ Skipped sample {i}: {e}")
            continue

    print(f"✅ Successfully formatted {len(formatted_data)} samples with metadata")
    return formatted_data

def create_data_collator(processor):
    """Create data collator (exactly from notebook)"""
    def collate_fn(examples):
        # Get the texts and images, and apply the chat template
        texts = [processor.apply_chat_template(example, tokenize=False) for example in examples]
        image_inputs = [process_vision_info(example)[0] for example in examples]

        # Tokenize the texts and process the images
        batch = processor(text=texts, images=image_inputs, return_tensors="pt", padding=True)

        # The labels are the input_ids, and we mask the padding tokens in the loss computation
        labels = batch["input_ids"].clone()
        labels[labels == processor.tokenizer.pad_token_id] = -100

        # Ignore the image token index in the loss computation (model specific)
        if isinstance(processor, Qwen2VLProcessor):
            image_tokens = [151652, 151653, 151655]  # Specific image token IDs for Qwen2VLProcessor
        else:
            image_tokens = [processor.tokenizer.convert_tokens_to_ids(processor.image_token)]

        # Mask image token IDs in the labels
        for image_token_id in image_tokens:
            labels[labels == image_token_id] = -100

        batch["labels"] = labels
        return batch

    return collate_fn

def fine_tune_weather_model(dataset_path, output_dir="qwen2-vl-weather-finetuned", max_samples=None):
    """Fine-tune Qwen2-VL for weather analysis using LoRA with full metadata"""

    print("🚀 Starting Qwen2-VL Weather Fine-tuning with METADATA")
    print("=" * 60)

    # Configuration
    model_id = "Qwen/Qwen2-VL-7B-Instruct"

    # Setup W&B
    try:
        wandb.init(
            project="qwen2-vl-weather-analysis-metadata",
            name="weather-satellite-metadata-lora",
            config={
                "model": model_id,
                "technique": "LoRA",
                "epochs": 3,  # Increased for full dataset
                "batch_size": 1,
                "learning_rate": 2e-4,
                "samples": max_samples or "ALL",
                "metadata_integration": True
            }
        )
        print("✅ W&B initialized")
    except Exception as e:
        print(f"⚠️ W&B failed, continuing without logging: {e}")
        os.environ["WANDB_MODE"] = "disabled"

    # Clear memory
    clear_memory()

    # Load dataset with metadata integration
    print("\n📂 Loading dataset with metadata integration...")
    dataset = load_weather_dataset(dataset_path, max_samples=max_samples)

    # Split dataset
    train_size = int(0.8 * len(dataset))
    train_dataset = dataset[:train_size]
    eval_dataset = dataset[train_size:]

    print(f"📊 Dataset split: {len(train_dataset)} train, {len(eval_dataset)} eval")

    # Log dataset info to W&B
    try:
        wandb.log({
            "dataset/total_samples": len(dataset),
            "dataset/train_samples": len(train_dataset),
            "dataset/eval_samples": len(eval_dataset),
            "dataset/metadata_integrated": True
        })
    except:
        pass

    # Setup quantization (exactly from notebook)
    print("\n🔧 Setting up model with quantization...")
    bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_use_double_quant=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.bfloat16
    )

    # Load model and processor
    model = Qwen2VLForConditionalGeneration.from_pretrained(
        model_id,
        device_map="auto",
        torch_dtype=torch.bfloat16,
        quantization_config=bnb_config
    )
    processor = Qwen2VLProcessor.from_pretrained(model_id)
    

    print("✅ Model and processor loaded")

    # Configure LoRA (exactly from notebook)
    print("\n⚙️ Setting up LoRA...")
    peft_config = LoraConfig(
        lora_alpha=16,
        lora_dropout=0.05,
        r=8,
        bias="none",
        target_modules=["q_proj", "v_proj"],
        task_type="CAUSAL_LM",
    )

    # Apply PEFT model adaptation
    peft_model = get_peft_model(model, peft_config)
    peft_model.print_trainable_parameters()

    # Log model info
    trainable_params = sum(p.numel() for p in peft_model.parameters() if p.requires_grad)
    total_params = sum(p.numel() for p in peft_model.parameters())

    try:
        wandb.log({
            "model/total_parameters": total_params,
            "model/trainable_parameters": trainable_params,
            "model/trainable_percentage": 100 * trainable_params / total_params,
            "model/lora_rank": 8,
            "model/lora_alpha": 16
        })
    except:
        pass

    # Training arguments (optimized for full dataset)
    print("\n🏋️ Setting up training...")
    epochs = 3 if len(dataset) > 500 else 2  # More epochs for larger dataset

    training_args = SFTConfig(
        output_dir=output_dir,
        num_train_epochs=epochs,
        per_device_train_batch_size=1,
        per_device_eval_batch_size=1,
        gradient_accumulation_steps=8,
        gradient_checkpointing=True,

        # Optimizer settings
        optim="adamw_torch",
        learning_rate=2e-4,
        lr_scheduler_type="constant",

        # Logging and evaluation (adjusted for larger dataset)
        logging_steps=10,
        eval_steps=max(50, len(train_dataset) // 20),  # Evaluate 20 times per epoch
        eval_strategy="steps",
        save_strategy="steps",
        save_steps=max(100, len(train_dataset) // 10),  # Save 10 times per epoch
        metric_for_best_model="eval_loss",
        greater_is_better=False,
        load_best_model_at_end=True,

        # Mixed precision
        bf16=True,
        tf32=False,
        max_grad_norm=0.3,
        warmup_ratio=0.03,

        # Reporting
        push_to_hub=False,
        report_to="wandb" if "WANDB_MODE" not in os.environ else "none",

        # Dataset configuration
        gradient_checkpointing_kwargs={"use_reentrant": False},
        dataset_text_field="",
        dataset_kwargs={"skip_prepare_dataset": True},
    )

    training_args.remove_unused_columns = False

    # Create data collator
    collate_fn = create_data_collator(processor)

    # Create trainer (exactly from notebook)
    trainer = SFTTrainer(
        model=peft_model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=eval_dataset,
        data_collator=collate_fn,
        peft_config=peft_config,
        processing_class=processor.tokenizer,
    )

    # Train the model
    print(f"\n🚀 Starting training for {epochs} epochs on {len(train_dataset)} samples...")
    trainer.train()

    # Save the model
    print("\n💾 Saving model...")
    trainer.save_model(output_dir)

    print(f"✅ Training completed! Model saved to: {output_dir}")

    # Log final metrics
    try:
        wandb.log({
            "training/completed": True,
            "training/final_model_path": output_dir,
            "training/epochs_completed": epochs,
            "training/samples_processed": len(train_dataset) * epochs
        })
    except:
        pass

    return output_dir

def test_finetuned_model(model_path, dataset_path):
    """Test the fine-tuned model (exactly from notebook)"""

    print(f"\n🧪 Testing fine-tuned model from: {model_path}")

    clear_memory()

    # Load base model
    model_id = "Qwen/Qwen2-VL-7B-Instruct"
    model = Qwen2VLForConditionalGeneration.from_pretrained(
        model_id,
        device_map="auto",
        torch_dtype=torch.bfloat16,
    )
    processor = Qwen2VLProcessor.from_pretrained(model_id)

    # Load adapter
    model.load_adapter(model_path)
    print("✅ Adapter loaded")

    # Load test data
    test_data = load_weather_dataset(dataset_path, max_samples=3)

    def generate_text_from_sample(model, processor, sample, max_new_tokens=512, device="cuda"):
        """Generate text from sample (exactly from notebook)"""
        # Prepare the text input by applying the chat template
        text_input = processor.apply_chat_template(
            sample[1:2],  # Use the sample without the system message
            tokenize=False,
            add_generation_prompt=True
        )

        # Process the visual input from the sample
        image_inputs, _ = process_vision_info(sample)

        # Prepare the inputs for the model
        model_inputs = processor(
            text=[text_input],
            images=image_inputs,
            return_tensors="pt",
        ).to(device)

        # Generate text with the model
        generated_ids = model.generate(**model_inputs, max_new_tokens=max_new_tokens)

        # Trim the generated ids to remove the input ids
        trimmed_generated_ids = [
            out_ids[len(in_ids):] for in_ids, out_ids in zip(model_inputs.input_ids, generated_ids)
        ]

        # Decode the output text
        output_text = processor.batch_decode(
            trimmed_generated_ids,
            skip_special_tokens=True,
            clean_up_tokenization_spaces=False
        )

        return output_text[0]

    # Test samples
    for i, sample in enumerate(test_data):
        print(f"\n📸 Testing sample {i+1}:")
        try:
            output = generate_text_from_sample(model, processor, sample)
            print(f"✅ Generated: {output[:200]}...")

            # Log to W&B
            try:
                wandb.log({
                    f"test_sample_{i}_output": output[:500],
                    f"test_sample_{i}_success": True
                })
            except:
                pass

        except Exception as e:
            print(f"❌ Test failed: {e}")

def main():
    """Main function"""

    print("🌍 QWEN2-VL WEATHER SATELLITE FINE-TUNING WITH METADATA")
    print("=" * 70)

    # Configuration - UPDATE THESE PATHS
    dataset_path = "/kaggle/input/wsi-data-pfe/wsi-data-r"  # Your dataset path
    output_dir = "./qwen2-vl-weather-finetuned-metadata"

    # IMPORTANT: Set max_samples to None to use ALL your data
    max_samples = None  # Use ALL 928+ samples
    # max_samples = 100  # Uncomment this line to test with 100 samples first

    print(f"📁 Dataset path: {dataset_path}")
    print(f"💾 Output directory: {output_dir}")
    print(f"📊 Max samples: {max_samples or 'ALL SAMPLES'}")

    # Check dataset path
    if not os.path.exists(dataset_path):
        print(f"❌ Dataset path not found: {dataset_path}")
        print("Please update the dataset_path variable with your actual dataset location")
        return

    try:
        # Fine-tune the model with metadata integration
        print("\n🔥 Starting fine-tuning with METADATA integration...")
        print("   - Weather data tables")
        print("   - City coordinates")
        print("   - Temporal context")
        print("   - Same prompts as GPT-4V collection")

        model_path = fine_tune_weather_model(
            dataset_path=dataset_path,
            output_dir=output_dir,
            max_samples=max_samples
        )

        # Test the fine-tuned model
        print("\n🧪 Testing fine-tuned model...")

        # Test the fine-tuned model
        test_finetuned_model(model_path, dataset_path)

        print("\n🎉 FINE-TUNING COMPLETED SUCCESSFULLY!")
        print(f"📁 Model saved at: {model_path}")

        # Final W&B cleanup
        try:
            wandb.finish()
        except:
            pass

    except Exception as e:
        print(f"\n❌ FINE-TUNING FAILED: {e}")
        import traceback
        traceback.print_exc()

    finally:
        clear_memory()



### Weights & Biases Integration
Set up W&B for experiment tracking and monitoring.

In [ ]:
import os
import wandb

# Set the WANDB API key
os.environ["WANDB_API_KEY"] = "YOUR_WANDB_TOKEN"

# Now login using wandb
wandb.login()

wandb: Currently logged in as: azdinsahir11 (azdinsahir11-university-mohamed-v) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

### Execute Fine-Tuning
Run the main training pipeline.

In [ ]:
if __name__ == "__main__":
    # Set environment variables
    os.environ["TOKENIZERS_PARALLELISM"] = "false"

    # Run fine-tuning
    main()

🌍 QWEN2-VL WEATHER SATELLITE FINE-TUNING WITH METADATA
📁 Dataset path: /kaggle/input/wsi-data-pfe/wsi-data-r
💾 Output directory: ./qwen2-vl-weather-finetuned-metadata
📊 Max samples: ALL SAMPLES

🔥 Starting fine-tuning with METADATA integration...
   - Weather data tables
   - City coordinates
   - Temporal context
   - Same prompts as GPT-4V collection
🚀 Starting Qwen2-VL Weather Fine-tuning with METADATA


✅ W&B initialized
GPU allocated memory: 0.00 GB
GPU reserved memory: 0.00 GB

📂 Loading dataset with metadata integration...
📂 Loading weather dataset from: /kaggle/input/wsi-data-pfe/wsi-data-r
Using ALL 973 samples
Processing 973 samples with metadata integration...
  Processed 0/973 samples...
  Processed 50/973 samples...
  Processed 100/973 samples...
  Processed 150/973 samples...
  Processed 200/973 samples...
  Processed 250/973 samples...
  Processed 300/973 samples...
  Processed 350/973 samples...
  Processed 400/973 samples...
  Processed 450/973 samples...
  Processed 500/973 samples...
  Processed 550/973 samples...
  Processed 600/973 samples...
  Processed 650/973 samples...
  Processed 700/973 samples...
  Processed 750/973 samples...
  Processed 800/973 samples...
  Processed 850/973 samples...
  Processed 900/973 samples...
  Processed 950/973 samples...
✅ Successfully formatted 973 samples with metadata
📊 Dataset split: 778 train, 195 eval

🔧 Setting up model with q

config.json: 0.00B [00:00, ?B/s]

model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

...


💾 Saving model...
✅ Training completed! Model saved to: ./qwen2-vl-weather-finetuned-metadata

🧪 Testing fine-tuned model...

🧪 Testing fine-tuned model from: ./qwen2-vl-weather-finetuned-metadata
GPU allocated memory: 0.02 GB
GPU reserved memory: 7.72 GB


Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

✅ Adapter loaded
📂 Loading weather dataset from: /kaggle/input/wsi-data-pfe/wsi-data-r
Limited to 3 samples (max_samples=3)
Processing 3 samples with metadata integration...
  Processed 0/3 samples...
✅ Successfully formatted 3 samples with metadata

📸 Testing sample 1:
✅ Generated: This MODIS Terra satellite image of Morocco provides a comprehensive view of the region's meteorological conditions, including cloud patterns, temperature distribution, and atmospheric moisture levels...

📸 Testing sample 2:
✅ Generated: This MODIS Terra satellite image of Morocco provides a comprehensive view of the region's meteorological conditions on the specified date. The image captures the following key atmospheric features:

1...

📸 Testing sample 3:
✅ Generated: This MODIS Terra satellite image of Morocco provides a comprehensive view of the region's meteorological conditions, including temperature, humidity, and cloud patterns. Here is a detailed analysis ba...

🎉 FINE-TUNING COMPLETED SUCCESSFULL

dataset/eval_samples,▁
dataset/total_samples,▁
dataset/train_samples,▁
eval/loss,█▃▂▁▁
eval/mean_token_accuracy,▁▆▇██
eval/num_tokens,▁▃▅▆█
eval/runtime,▁▆▁█▅
eval/samples_per_second,▁▁▁▁▁
eval/steps_per_second,▁▁▁▁▁
model/lora_alpha,▁
model/lora_rank,▁


GPU allocated memory: 0.02 GB
GPU reserved memory: 7.72 GB


## 4. Model Evaluation & Testing

### Test the Fine-Tuned Model
Evaluate the fine-tuned model on test samples.

In [ ]:
test_finetuned_model(model_path, dataset_path)

print("\n🎉 FINE-TUNING COMPLETED SUCCESSFULLY!")
print(f"📁 Model saved at: {model_path}")


## 5. Model Export & Deployment

### Upload to HuggingFace Hub
Export and share your fine-tuned model on HuggingFace.

#### Step 1: Login to HuggingFace
First, authenticate with your HuggingFace account using a write-access token.  
Get your token from: https://huggingface.co/settings/tokens

In [ ]:
from huggingface_hub import login
login(token="YOUR_HF_TOKEN")  # Replace with WRITE token
print("✅ Logged in with WRITE permissions!")

✅ Logged in with WRITE permissions!


#### Step 2: Upload Model to Hub
Upload all model files to your HuggingFace repository.

In [ ]:
from huggingface_hub import HfApi

model_path = "/content/qwen2-vl-weather-finetuned-metadata"
hf_repo_name = "azdin/qwen2-vl-weather-satellite"

api = HfApi()

try:
    # Create repository with write token
    api.create_repo(
        repo_id=hf_repo_name,
        exist_ok=True,
        token = hf_token,
        private=False,  # Set to True for private repo
        repo_type="model"
    )
    print(f"✅ Repository created: {hf_repo_name}")

    # Upload all files
    api.upload_folder(
        folder_path=model_path,
        repo_id=hf_repo_name,
        commit_message="Upload Qwen2-VL weather satellite LoRA adapter"
    )

    print(f"✅ Model uploaded successfully!")
    print(f"🔗 View at: https://huggingface.co/{hf_repo_name}")

except Exception as e:
    print(f"❌ Upload failed: {e}")
    if "403" in str(e):
        print("💡 Make sure your token has WRITE permissions!")
    elif "401" in str(e):
        print("💡 Check if your token is correct!")


✅ Repository created: azdin/qwen2-vl-weather-satellite
❌ Upload failed: (Request ID: Root=1-687f9497-74ba1e783be95342046165b4;bd3f50c7-5d1d-4a39-b9fb-e368ec71813b)

403 Forbidden: You have read access but not the required permissions for this operation.
Cannot access content at: https://huggingface.co/azdin/qwen2-vl-weather-satellite.git/info/lfs/objects/batch.
Make sure your token has the correct permissions.
💡 Make sure your token has WRITE permissions!


---

## 6. Advanced PEFT Techniques (Optional)

### DoRA (Weight-Decomposed Low-Rank Adaptation)
An enhanced version of LoRA that decomposes weights for better performance.

In [ ]:
# ====================================================================
# CELL 5: DoRA Technique Training
# ====================================================================

print("\n" + "="*60)
print("🔥 TECHNIQUE 1: DoRA (Weight-Decomposed LoRA)")
print("="*60)

# Setup W&B for DoRA
dora_config = {
    "technique": "DoRA",
    "quantization": "4-bit NF4",
    "lora_rank": 16,
    "lora_alpha": 32,
    "target_modules": ["q_proj", "v_proj", "k_proj", "o_proj"],
    "samples": MAX_SAMPLES,
    "epochs": 3
}

wandb.init(
    project="qwen2vl-dora-weather",
    name="weather-satellite-dora",
    config=dora_config,
    tags=["dora", "weather-satellite", "comparison"]
)

clear_memory()

# Setup quantization
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

# Load model
print("📥 Loading model for DoRA...")
model = Qwen2VLForConditionalGeneration.from_pretrained(
    MODEL_ID,
    device_map="auto",
    torch_dtype=torch.bfloat16,
    quantization_config=bnb_config
)
processor = Qwen2VLProcessor.from_pretrained(MODEL_ID)

# DoRA config
try:
    peft_config = LoraConfig(
        lora_alpha=32,
        lora_dropout=0.05,
        r=16,
        bias="none",
        target_modules=["q_proj", "v_proj", "k_proj", "o_proj"],
        task_type="CAUSAL_LM",
        use_dora=True,  # Enable DoRA
        use_rslora=True,
    )
    print("✅ DoRA configuration created")
except Exception as e:
    print(f"⚠️ DoRA not supported, using enhanced LoRA: {e}")
    peft_config = LoraConfig(
        lora_alpha=32,
        lora_dropout=0.05,
        r=16,
        bias="none",
        target_modules=["q_proj", "v_proj", "k_proj", "o_proj"],
        task_type="CAUSAL_LM",
        use_rslora=True,
    )

# Apply PEFT
peft_model = get_peft_model(model, peft_config)
peft_model.print_trainable_parameters()

# Training arguments for DoRA
dora_output_dir = "./dora_weather_model"
training_args = SFTConfig(
    output_dir=dora_output_dir,
    num_train_epochs=3,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    gradient_accumulation_steps=8,
    gradient_checkpointing=True,

    optim="adamw_torch",
    learning_rate=1e-4,  # Lower LR for DoRA
    lr_scheduler_type="constant",

    logging_steps=10,
    eval_steps=max(25, len(train_dataset) // 30),
    eval_strategy="steps",
    save_strategy="steps",
    save_steps=max(50, len(train_dataset) // 20),
    metric_for_best_model="eval_loss",
    greater_is_better=False,
    load_best_model_at_end=True,

    bf16=True,
    tf32=False,
    max_grad_norm=0.3,
    warmup_ratio=0.03,

    push_to_hub=False,
    report_to="wandb",

    gradient_checkpointing_kwargs={"use_reentrant": False},
    dataset_text_field="",
    dataset_kwargs={"skip_prepare_dataset": True},
)

training_args.remove_unused_columns = False

# Create trainer
trainer = SFTTrainer(
    model=peft_model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    data_collator=create_data_collator(processor),
    peft_config=peft_config,
    processing_class=processor.tokenizer,
)

# Train DoRA
print("🚀 Starting DoRA training...")
dora_start_time = time.time()

trainer.train()

dora_training_time = time.time() - dora_start_time

# Save model locally
trainer.save_model(dora_output_dir)

# Final evaluation
dora_eval_result = trainer.evaluate()

print(f"✅ DoRA training completed in {dora_training_time/60:.1f} minutes")
print(f"📊 Final DoRA eval loss: {dora_eval_result['eval_loss']:.4f}")

# Upload to HuggingFace
dora_hf_repo = upload_to_huggingface(dora_output_dir, "qwen2-vl-weather-dora", "DoRA")

# Log final results
wandb.log({
    "final_training_time": dora_training_time,
    "final_eval_loss": dora_eval_result['eval_loss'],
    "hf_repo": dora_hf_repo,
    "training_completed": True
})

wandb.finish()

print(f"💾 DoRA model saved locally: {dora_output_dir}")
if dora_hf_repo:
    print(f"🌐 DoRA model on HF: https://huggingface.co/{dora_hf_repo}")



🔥 TECHNIQUE 1: DoRA (Weight-Decomposed LoRA)


wandb: Currently logged in as: azdinsahir11 (azdinsahir11-university-mohamed-v) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


GPU allocated: 0.00 GB
GPU reserved: 0.00 GB
📥 Loading model for DoRA...


model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

...

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

training_args.bin:   0%|          | 0.00/5.75k [00:00<?, ?B/s]

✅ Successfully uploaded to: https://huggingface.co/azdin/qwen2-vl-weather-dora


eval/loss,█▄▃▂▂▂▁▁▁▁▁▁
eval/mean_token_accuracy,▁▅▆▆▇▇▇█████
eval/num_tokens,▁▂▂▃▄▄▅▆▆▇██
eval/runtime,▅▃▃▁▆▄█▇▅▇█▇
eval/samples_per_second,▃▆▅█▃▅▁▃▃▃▁▁
eval/steps_per_second,▃▆▅█▃▅▁▃▃▃▁▁
final_eval_loss,▁
final_training_time,▁
train/epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇████
train/global_step,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇█████
train/grad_norm,█▂▂▂▂▂▁▁▁▁▁▂▁▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁


💾 DoRA model saved locally: ./dora_weather_model
🌐 DoRA model on HF: https://huggingface.co/azdin/qwen2-vl-weather-dora


### AdaLoRA (Adaptive Low-Rank Adaptation)
Adaptive approach that dynamically adjusts the rank allocation during training.

In [ ]:
# ====================================================================
# CELL 6: AdaLoRA Technique Training
# ====================================================================

print("\n" + "="*60)
print("🔥 TECHNIQUE 2: AdaLoRA (Adaptive LoRA)")
print("="*60)

# Setup W&B for AdaLoRA
adalora_config = {
    "technique": "AdaLoRA",
    "quantization": "4-bit NF4",
    "init_r": 12,
    "target_r": 8,
    "beta1": 0.85,
    "beta2": 0.85,
    "tinit": 200,
    "tfinal": 1000,
    "samples": MAX_SAMPLES,
    "epochs": 4
}

wandb.init(
    project="qwen2vl-adalora-weather",
    name="weather-satellite-adalora",
    config=adalora_config,
    tags=["adalora", "weather-satellite", "comparison"]
)

clear_memory()

# Setup quantization
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

# Load model for AdaLoRA
print("📥 Loading model for AdaLoRA...")
model = Qwen2VLForConditionalGeneration.from_pretrained(
    MODEL_ID,
    device_map="auto",
    torch_dtype=torch.bfloat16,
    quantization_config=bnb_config
)
processor = Qwen2VLProcessor.from_pretrained(MODEL_ID)

# AdaLoRA config
try:
    peft_config = AdaLoraConfig(
        init_r=12,
        target_r=8,
        beta1=0.85,
        beta2=0.85,
        tinit=200,
        tfinal=1000,
        deltaT=10,
        lora_alpha=16,
        lora_dropout=0.05,
        target_modules=["q_proj", "v_proj", "k_proj", "o_proj"],
        task_type="CAUSAL_LM",
    )
    print("✅ AdaLoRA configuration created")
except Exception as e:
    print(f"⚠️ AdaLoRA not available, using adaptive LoRA: {e}")
    peft_config = LoraConfig(
        lora_alpha=24,
        lora_dropout=0.1,
        r=12,
        bias="none",
        target_modules=["q_proj", "v_proj", "k_proj", "o_proj"],
        task_type="CAUSAL_LM",
    )

# Apply PEFT
peft_model = get_peft_model(model, peft_config)
peft_model.print_trainable_parameters()

# Training arguments for AdaLoRA
adalora_output_dir = "./adalora_weather_model"
training_args = SFTConfig(
    output_dir=adalora_output_dir,
    num_train_epochs=4,  # More epochs for adaptation
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    gradient_accumulation_steps=8,
    gradient_checkpointing=True,

    optim="adamw_torch",
    learning_rate=3e-4,  # Higher LR for AdaLoRA
    lr_scheduler_type="cosine",  # Cosine schedule for adaptive learning

    logging_steps=10,
    eval_steps=max(20, len(train_dataset) // 40),
    eval_strategy="steps",
    save_strategy="steps",
    save_steps=max(40, len(train_dataset) // 25),
    metric_for_best_model="eval_loss",
    greater_is_better=False,
    load_best_model_at_end=True,

    bf16=True,
    tf32=False,
    max_grad_norm=0.3,
    warmup_ratio=0.1,  # More warmup for adaptation

    push_to_hub=False,
    report_to="wandb",

    gradient_checkpointing_kwargs={"use_reentrant": False},
    dataset_text_field="",
    dataset_kwargs={"skip_prepare_dataset": True},
)

training_args.remove_unused_columns = False

# Create trainer
trainer = SFTTrainer(
    model=peft_model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    data_collator=create_data_collator(processor),
    peft_config=peft_config,
    processing_class=processor.tokenizer,
)

# Train AdaLoRA
print("🚀 Starting AdaLoRA training...")
adalora_start_time = time.time()

trainer.train()

adalora_training_time = time.time() - adalora_start_time

# Save model locally
trainer.save_model(adalora_output_dir)

# Final evaluation
adalora_eval_result = trainer.evaluate()

print(f"✅ AdaLoRA training completed in {adalora_training_time/60:.1f} minutes")
print(f"📊 Final AdaLoRA eval loss: {adalora_eval_result['eval_loss']:.4f}")

# Upload to HuggingFace
adalora_hf_repo = upload_to_huggingface(adalora_output_dir, "qwen2-vl-weather-adalora", "AdaLoRA")

# Log final results
wandb.log({
    "final_training_time": adalora_training_time,
    "final_eval_loss": adalora_eval_result['eval_loss'],
    "hf_repo": adalora_hf_repo,
    "training_completed": True
})

wandb.finish()

print(f"💾 AdaLoRA model saved locally: {adalora_output_dir}")
if adalora_hf_repo:
    print(f"🌐 AdaLoRA model on HF: https://huggingface.co/{adalora_hf_repo}")




🔥 TECHNIQUE 2: AdaLoRA (Adaptive LoRA)


GPU allocated: 0.00 GB
GPU reserved: 0.00 GB
📥 Loading model for AdaLoRA...


config.json: 0.00B [00:00, ?B/s]

model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

...

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

training_args.bin:   0%|          | 0.00/5.75k [00:00<?, ?B/s]

✅ Successfully uploaded to: https://huggingface.co/azdin/qwen2-vl-weather-adalora


eval/loss,█▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
eval/mean_token_accuracy,▁▇▇█████████████████
eval/num_tokens,▁▁▂▂▃▃▃▄▄▄▅▅▆▆▆▇▇▇██
eval/runtime,▅▆▆▅▁▃▆▆█▆▆▆▄▇▆▅▄▃▃▃
eval/samples_per_second,▄▃▄▄█▆▃▃▁▃▃▃▅▃▃▄▄▆▆▆
eval/steps_per_second,▄▃▄▄█▆▃▃▁▃▃▃▅▃▃▄▄▆▆▆
final_eval_loss,▁
final_training_time,▁
train/epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▄▄▄▄▄▅▅▅▅▆▆▆▆▆▆▆▇▇▇▇▇▇█████
train/global_step,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇█████
train/grad_norm,▂▄▆▂▂▂█▄▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁


💾 AdaLoRA model saved locally: ./adalora_weather_model
🌐 AdaLoRA model on HF: https://huggingface.co/azdin/qwen2-vl-weather-adalora


---

## Summary & Next Steps

### What We Accomplished
✅ Fine-tuned Qwen2-VL model for weather satellite image analysis  
✅ Implemented LoRA for parameter-efficient training  
✅ Evaluated model performance on test data  
✅ Uploaded model to HuggingFace Hub  


### Resources
- [Qwen2-VL Documentation](https://github.com/QwenLM/Qwen2-VL)
- [PEFT Documentation](https://huggingface.co/docs/peft)
- [TRL Documentation](https://huggingface.co/docs/trl)

---
**Author**: Azeddin sahir 